In [ ]:
!pip -q install xarray netCDF4 h5netcdf cftime dask[complete] \
                 rioxarray rasterio leafmap folium \
                 numpy pandas tqdm imageio \
                 torch torchvision torchaudio \
                 pyngrok streamlit xskillscore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.8/600.8 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 153.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.2 

In [ ]:
!pip -q install localtileserver matplotlib pillow

In [ ]:
import os, numpy as np, pandas as pd, xarray as xr, dask, torch, platform, gc
from pathlib import Path

# Your uploaded files
PATH_INST = "/content/data_stream-oper_stepType-instant.nc"
PATH_ACC  = "/content/data_stream-oper_stepType-accum.nc"

OUT_DIR = "/content/neural_weather_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# Keep memory stable while still training a strong model
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
dask.config.set({"array.slicing.split_large_chunks": True})
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

CFG = {
    "tin": 6,                   # past hours as input
    "lead_train": 1,            # train for +1h
    "batch_size": 2,            # keep small; we'll use grad accumulation
    "accum_steps": 4,           # effective batch = batch_size * accum_steps
    "epochs": 10,               # increase if you have time
    "lr": 3e-4,
    "num_workers": 0,           # single-process loader (most stable in Colab)
    "seed": 42,
    "patch_hw": 96,             # train on 96x96 patches (stronger than tiny patches)
    "samples_per_epoch": 1500,  # many random samples per epoch
    "tile_patch": 192,          # tile size at inference
    "tile_overlap": 32,         # overlap to blend tiles
}
CFG

{'tin': 6,
 'lead_train': 1,
 'batch_size': 2,
 'accum_steps': 4,
 'epochs': 10,
 'lr': 0.0003,
 'num_workers': 0,
 'seed': 42,
 'patch_hw': 96,
 'samples_per_epoch': 1500,
 'tile_patch': 192,
 'tile_overlap': 32}

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def open_nc_safe(path):
    for eng in ("netcdf4", "h5netcdf", "scipy"):
        try:
            ds = xr.open_dataset(path, engine=eng, chunks={"valid_time": 24, "time": 24})
            v0 = list(ds.data_vars)[0]
            idx = {d: 0 for d in ds[v0].dims}
            _ = ds[v0].isel(**idx).values
            print(f"Opened {path} with engine={eng}")
            return ds
        except Exception as e:
            print(f"Engine {eng} failed for {path}: {e}")
    raise RuntimeError(f"Failed to open {path} with any engine")

def harmonize_era5(ds):
    if "lon" in ds.coords: ds = ds.rename({"lon": "longitude"})
    if "lat" in ds.coords: ds = ds.rename({"lat": "latitude"})
    if "valid_time" in ds.coords and "time" not in ds.coords:
        ds = ds.rename({"valid_time": "time"})
    if "step" in ds.dims:  # keep last step (hourly)
        ds = ds.isel(step=-1).drop_vars("step", errors="ignore")
    if "expver" in ds.dims:
        ds = ds.isel(expver=0, drop=True)
    return ds

dsi = open_nc_safe(PATH_INST)
dsa = open_nc_safe(PATH_ACC)
dsi = harmonize_era5(dsi)
dsa = harmonize_era5(dsa)

# Rename short codes to long names if present
rename_inst = {}
if "t2m" in dsi: rename_inst["t2m"] = "2m_temperature"
if "d2m" in dsi: rename_inst["d2m"] = "2m_dewpoint_temperature"
if "u10" in dsi: rename_inst["u10"] = "10m_u_component_of_wind"
if "v10" in dsi: rename_inst["v10"] = "10m_v_component_of_wind"
if "msl" in dsi: rename_inst["msl"] = "mean_sea_level_pressure"
if "tcc" in dsi: rename_inst["tcc"] = "total_cloud_cover"
if "tcwv" in dsi: rename_inst["tcwv"] = "total_column_water_vapour"
if "cape" in dsi: rename_inst["cape"] = "convective_available_potential_energy"
dsi = dsi.rename(rename_inst)

if "tp" in dsa:
    dsa = dsa.rename({"tp": "total_precipitation"})

print("INST vars:", list(dsi.data_vars)[:10])
print("ACC  vars:", list(dsa.data_vars)[:10])

Opened /content/data_stream-oper_stepType-instant.nc with engine=netcdf4
Opened /content/data_stream-oper_stepType-accum.nc with engine=netcdf4
INST vars: ['10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature', 'mean_sea_level_pressure', 'total_cloud_cover', 'tcw', 'convective_available_potential_energy']
ACC  vars: ['total_precipitation']


In [ ]:
# Predictors (use all strong ones if present)
keep_vars = [
    "2m_temperature","2m_dewpoint_temperature",
    "10m_u_component_of_wind","10m_v_component_of_wind",
    "mean_sea_level_pressure","total_cloud_cover",
    "total_column_water_vapour","convective_available_potential_energy"
]
inst = xr.Dataset({v: dsi[v] for v in keep_vars if v in dsi})

# Unit conversions (lazy)
if "2m_temperature" in inst: inst["2m_temperature"] = inst["2m_temperature"] - 273.15
if "2m_dewpoint_temperature" in inst: inst["2m_dewpoint_temperature"] = inst["2m_dewpoint_temperature"] - 273.15
if "mean_sea_level_pressure" in inst: inst["mean_sea_level_pressure"] = inst["mean_sea_level_pressure"] / 100.0

# Precip: m -> mm/h (hourly accumulation)
assert "total_precipitation" in dsa, "total_precipitation not found in accum file"
tp_mmph = (dsa["total_precipitation"] * 1000.0).rename("tp_mmph")

# Merge by common coords
ds = xr.merge([inst, tp_mmph.to_dataset()], join="inner")
lat = ds["latitude"].values
lon = ds["longitude"].values
print("Grid:", len(lat), "x", len(lon), "| Time:", ds.time.size)
print("Merged vars:", list(ds.data_vars))

Grid: 125 x 121 | Time: 720
Merged vars: ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', 'total_cloud_cover', 'convective_available_potential_energy', 'tp_mmph']


In [ ]:
train_ds = ds.sel(time=slice("2025-04-01", "2025-04-20"))
val_ds   = ds.sel(time=slice("2025-04-21", "2025-04-26"))
test_ds  = ds.sel(time=slice("2025-04-27", "2025-04-30"))
print(train_ds.time.size, val_ds.time.size, test_ds.time.size)

480 144 96


In [ ]:
preds_avail = [v for v in keep_vars if v in train_ds]
# Sample every 6th hour for stats (good enough)
Xsample = train_ds[preds_avail].isel(time=slice(0, None, 6)).to_array("channel")
mean = Xsample.mean(dim=("time","latitude","longitude"))
std  = Xsample.std(dim=("time","latitude","longitude")) + 1e-6

# Store as numpy aligned to the same order
mean_np = mean.values.astype(np.float32)
std_np  = std.values.astype(np.float32)
print("Channels:", list(mean.channel.values))

Channels: ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', 'total_cloud_cover', 'convective_available_potential_energy']


In [ ]:
# Repair cell: define device and rebuild training objects
import torch, gc
from torch.optim import AdamW
import torch.nn as nn

# 1) Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2) Sanity: ensure these exist (rerun your normalization cell if not)
assert 'preds_avail' in globals(), "Run the normalization cell to define preds_avail"
assert 'mean_np' in globals() and 'std_np' in globals(), "Run the normalization cell to define mean/std"
assert 'UNet3DNowcast' in globals(), "Run the model class cell first"
assert 'CFG' in globals(), "Run the config cell first"

# 3) (Re)build model + optimizer + loss + scaler
in_ch = len(preds_avail) + 4  # 4 time-feature channels
model = UNet3DNowcast(in_ch=in_ch, base=16).to(device)
optimizer = AdamW(model.parameters(), lr=CFG["lr"])
loss_fn = nn.SmoothL1Loss(beta=0.5)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

# 4) If you need DataLoaders again (after a restart), recreate them:
if 'train_loader' not in globals() or 'val_loader' not in globals():
    from torch.utils.data import DataLoader
    pin = False  # keep False for Colab stability
    train_loader = DataLoader(train_set, batch_size=CFG["batch_size"], shuffle=True,
                              num_workers=CFG["num_workers"], pin_memory=pin, persistent_workers=False)
    val_loader   = DataLoader(val_set,   batch_size=CFG["batch_size"], shuffle=False,
                              num_workers=CFG["num_workers"], pin_memory=pin, persistent_workers=False)

# 5) Optional: sanity check one batch (helps catch shape issues early)
xb, yb = next(iter(train_loader))
print("xb:", tuple(xb.shape), "yb:", tuple(yb.shape))  # expect [B, C+4, Tin, H, W] and [B, 1, H, W]

Using device: cuda
xb: (2, 10, 6, 96, 96) yb: (2, 1, 96, 96)


In [ ]:
import numpy as np, torch
from torch.utils.data import Dataset

def make_time_feats(times):
    t = pd.to_datetime(times)
    hod = t.hour + t.minute/60.0
    hod_sin = np.sin(2*np.pi*hod/24.0)
    hod_cos = np.cos(2*np.pi*hod/24.0)
    doy = t.dayofyear
    doy_sin = np.sin(2*np.pi*(doy-1)/365.0)
    doy_cos = np.cos(2*np.pi*(doy-1)/365.0)
    return np.stack([hod_sin, hod_cos, doy_sin, doy_cos], axis=1).astype(np.float32)  # [T,4]

class OnTheFlyNowcast(Dataset):
    def __init__(self, dsX: xr.Dataset, preds: list, target: str,
                 tin: int, lead: int,
                 mean_np: np.ndarray, std_np: np.ndarray,
                 patch_hw=96, mode="train", samples_per_epoch=1500, seed=42):
        self.dsX, self.preds, self.target = dsX, preds, target
        self.tin, self.lead = tin, lead
        self.mean_np, self.std_np = mean_np, std_np
        self.patch_hw, self.mode = patch_hw, mode
        self.rng = np.random.default_rng(seed)
        T = dsX.time.size
        self.valid_ends = np.arange(tin-1, T - lead, dtype=int)
        self.H = dsX.dims["latitude"]; self.W = dsX.dims["longitude"]
        self.N = samples_per_epoch if mode == "train" else len(self.valid_ends)

    def __len__(self): return self.N

    def _pick_end(self, idx):
        return int(self.rng.choice(self.valid_ends)) if self.mode=="train" else int(self.valid_ends[idx])

    def _pick_patch(self):
        ph = pw = self.patch_hw
        if ph is None or ph >= min(self.H, self.W):
            return 0, 0, self.H, self.W
        if self.mode == "train":
            top = self.rng.integers(0, self.H - ph + 1)
            left= self.rng.integers(0, self.W - pw + 1)
        else:
            top = (self.H - ph)//2; left = (self.W - pw)//2
        return top, left, ph, pw

    def __getitem__(self, idx):
        e = self._pick_end(idx)
        t_slice = slice(e-(self.tin-1), e+1)
        top, left, ph, pw = self._pick_patch()

        # Predictors: (C,T,H,W) as numpy
        X_da = self.dsX[self.preds].to_array("channel").isel(
            time=t_slice, latitude=slice(top, top+ph), longitude=slice(left, left+pw)
        ).transpose("channel","time","latitude","longitude")
        X_np = np.asarray(X_da.data, dtype=np.float32)
        # Normalize
        X_np = (X_np - self.mean_np[:, None, None, None]) / self.std_np[:, None, None, None]

        # Time features: [T,4] -> [4,T,1,1] -> broadcast to [4,T,ph,pw]
        times = self.dsX.time.isel(time=t_slice).values
        tf = make_time_feats(times)                  # [T,4]
        TF = np.transpose(tf, (1,0))                # [4,T]
        TF = TF[:, :, None, None]                   # [4,T,1,1]
        TF = np.broadcast_to(TF, (4, self.tin, ph, pw)).astype(np.float32)  # [4,T,ph,pw]

        # Stack features along channel dim: [C+4,T,ph,pw]
        X_np = np.concatenate([X_np, TF], axis=0)

        # Target at lead (log1p)
        y_da = self.dsX[self.target].isel(
            time=e+self.lead, latitude=slice(top, top+ph), longitude=slice(left, left+pw)
        )
        y_np = np.log1p(np.asarray(y_da.data, dtype=np.float32))  # [ph,pw]

        return torch.from_numpy(X_np), torch.from_numpy(y_np[None, ...])

In [ ]:
from torch.utils.data import DataLoader

train_set = OnTheFlyNowcast(
    dsX=train_ds, preds=preds_avail, target="tp_mmph",
    tin=CFG["tin"], lead=CFG["lead_train"],
    mean_np=mean_np, std_np=std_np,
    patch_hw=CFG["patch_hw"], mode="train",
    samples_per_epoch=CFG["samples_per_epoch"], seed=CFG["seed"]
)
val_set = OnTheFlyNowcast(
    dsX=val_ds, preds=preds_avail, target="tp_mmph",
    tin=CFG["tin"], lead=CFG["lead_train"],
    mean_np=mean_np, std_np=std_np,
    patch_hw=CFG["patch_hw"], mode="val",
    samples_per_epoch=min(len(val_ds.time), 300)
)

train_loader = DataLoader(train_set, batch_size=CFG["batch_size"], shuffle=True,
                          num_workers=CFG["num_workers"], pin_memory=False, persistent_workers=False)
val_loader   = DataLoader(val_set,   batch_size=CFG["batch_size"], shuffle=False,
                          num_workers=CFG["num_workers"], pin_memory=False, persistent_workers=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm3d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm3d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.net(x)

class Down3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.pool = nn.MaxPool3d((1,2,2))
        self.conv = DoubleConv3D(in_ch, out_ch)
    def forward(self, x): return self.conv(self.pool(x))

class Up3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose3d(in_ch, out_ch, kernel_size=(1,2,2), stride=(1,2,2))
        self.conv = DoubleConv3D(in_ch, out_ch)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size(3) - x1.size(3)
        diffX = x2.size(4) - x1.size(4)
        x1 = F.pad(x1, [diffX//2, diffX - diffX//2, diffY//2, diffY - diffY//2, 0,0])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet3DNowcast(nn.Module):
    def __init__(self, in_ch, base=16):
        super().__init__()
        self.inc   = DoubleConv3D(in_ch, base)
        self.down1 = Down3D(base, base*2)
        self.down2 = Down3D(base*2, base*4)
        self.up1   = Up3D(base*4, base*2)
        self.up2   = Up3D(base*2, base)
        self.out3d = nn.Conv3d(base, base, 3, padding=1)
        self.out2d = nn.Conv2d(base, 1, 1)

    def forward(self, x):  # [B,C,T,H,W]
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x  = self.up1(x3, x2)
        x  = self.up2(x,  x1)
        x  = self.out3d(x)       # [B,base,T,H,W]
        x  = torch.mean(x, 2)    # temporal pooling
        x  = self.out2d(x)       # [B,1,H,W] in log1p space
        return x

in_ch = len(preds_avail) + 4  # add 4 time features
model = UNet3DNowcast(in_ch=in_ch, base=16).to(device)
sum(p.numel() for p in model.parameters())/1e6

0.340465

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=CFG["lr"])
loss_fn = nn.SmoothL1Loss(beta=0.5)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

def metrics_mm(y_true_log, y_pred_log):
    yt = torch.expm1(y_true_log)
    yp = torch.expm1(y_pred_log).clamp(min=0)
    mae = torch.mean(torch.abs(yp - yt)).item()
    rmse = torch.sqrt(torch.mean((yp - yt)**2)).item()
    return mae, rmse

best_val = float("inf")
for epoch in range(CFG["epochs"]):
    model.train()
    trL=trMAE=trRMSE=0.0
    optimizer.zero_grad(set_to_none=True)
    for step, (xb, yb) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{CFG['epochs']} train", leave=False), 1):
        xb, yb = xb.to(device), yb.to(device)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            pred = model(xb)
            loss = loss_fn(pred, yb) / CFG["accum_steps"]
        scaler.scale(loss).backward()
        if step % CFG["accum_steps"] == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
        mae, rmse = metrics_mm(yb, pred)
        trL += loss.item()*CFG["accum_steps"]; trMAE += mae; trRMSE += rmse

    model.eval()
    vaL=vaMAE=vaRMSE=0.0
    with torch.no_grad():
        for xb, yb in tqdm(val_loader, desc=f"Epoch {epoch+1}/{CFG['epochs']} val", leave=False):
            xb, yb = xb.to(device), yb.to(device)
            with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
                pred = model(xb)
                loss = loss_fn(pred, yb)
            mae, rmse = metrics_mm(yb, pred)
            vaL += loss.item(); vaMAE += mae; vaRMSE += rmse

    print(f"Epoch {epoch+1:02d} | train L {trL/len(train_loader):.3f} MAE {trMAE/len(train_loader):.3f} RMSE {trRMSE/len(train_loader):.3f} | "
          f"val L {vaL/len(val_loader):.3f} MAE {vaMAE/len(val_loader):.3f} RMSE {vaRMSE/len(val_loader):.3f}")

    if (vaRMSE/len(val_loader)) < best_val:
        best_val = vaRMSE/len(val_loader)
        torch.save(model.state_dict(), os.path.join(OUT_DIR, "unet3d_best.pth"))
        print("  ✓ Saved best model")

    torch.cuda.empty_cache(); gc.collect()

print("Training done. Best val RMSE(mm):", best_val)

Epoch 01 | train L 0.022 MAE 0.095 RMSE 0.337 | val L 0.009 MAE 0.057 RMSE 0.226
  ✓ Saved best model


Epoch 02 | train L 0.018 MAE 0.083 RMSE 0.309 | val L 0.008 MAE 0.039 RMSE 0.218
  ✓ Saved best model


Epoch 03 | train L 0.016 MAE 0.078 RMSE 0.297 | val L 0.009 MAE 0.042 RMSE 0.225


Epoch 04 | train L 0.015 MAE 0.075 RMSE 0.286 | val L 0.008 MAE 0.041 RMSE 0.218


Epoch 05 | train L 0.014 MAE 0.073 RMSE 0.278 | val L 0.008 MAE 0.040 RMSE 0.214
  ✓ Saved best model


Epoch 06 | train L 0.014 MAE 0.071 RMSE 0.268 | val L 0.008 MAE 0.038 RMSE 0.213
  ✓ Saved best model


Epoch 07 | train L 0.013 MAE 0.068 RMSE 0.258 | val L 0.008 MAE 0.044 RMSE 0.216


Epoch 08 | train L 0.012 MAE 0.068 RMSE 0.255 | val L 0.008 MAE 0.051 RMSE 0.220


Epoch 09 | train L 0.011 MAE 0.063 RMSE 0.243 | val L 0.008 MAE 0.041 RMSE 0.214


Epoch 10 | train L 0.012 MAE 0.064 RMSE 0.243 | val L 0.008 MAE 0.042 RMSE 0.213
Training done. Best val RMSE(mm): 0.2127601852019628


In [ ]:
import numpy as np, xarray as xr, pandas as pd, torch, gc
from pathlib import Path
import rioxarray as rxr

# How many rainy cases to export
TOP_K = 12

def build_input_stack(dsX, preds, mean_np, std_np, t_start, tin):
    sel = dsX[preds].isel(time=slice(t_start, t_start+tin)).to_array("channel").transpose("channel","time","latitude","longitude")
    X = np.asarray(sel.data, dtype=np.float32)
    X = (X - mean_np[:, None, None, None]) / std_np[:, None, None, None]
    times = dsX.time.isel(time=slice(t_start, t_start+tin)).values
    # Time features → [4, T, H, W]
    def make_time_feats(times):
        t = pd.to_datetime(times)
        hod = t.hour + t.minute/60.0
        hod_sin = np.sin(2*np.pi*hod/24.0)
        hod_cos = np.cos(2*np.pi*hod/24.0)
        doy = t.dayofyear
        doy_sin = np.sin(2*np.pi*(doy-1)/365.0)
        doy_cos = np.cos(2*np.pi*(doy-1)/365.0)
        return np.stack([hod_sin, hod_cos, doy_sin, doy_cos], axis=1).astype(np.float32)
    tf = make_time_feats(times)                 # [T,4]
    TF = np.transpose(tf, (1,0))               # [4,T]
    H = dsX.dims["latitude"]; W = dsX.dims["longitude"]
    TF = TF[:, :, None, None]                  # [4,T,1,1]
    TF = np.broadcast_to(TF, (4, len(times), H, W)).astype(np.float32)
    return np.concatenate([X, TF], axis=0)     # [C+4, T, H, W]

def predict_full(model, X_full, patch=192, overlap=32):
    device = next(model.parameters()).device
    C,T,H,W = X_full.shape
    out = np.zeros((H,W), dtype=np.float32)
    wgt = np.zeros((H,W), dtype=np.float32)
    step = patch - overlap
    model.eval()
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
        for r in range(0, H, step):
            for c in range(0, W, step):
                rr = min(r+patch, H); cc = min(c+patch, W)
                r0 = rr - patch if rr - r < patch else r
                c0 = cc - patch if cc - c < patch else c
                x = X_full[:, :, r0:rr, c0:cc][None]
                y = model(torch.from_numpy(x).to(device)).float().cpu().numpy()[0,0]
                y_mm = np.expm1(y).clip(min=0)
                out[r0:rr, c0:cc] += y_mm
                wgt[r0:rr, c0:cc] += 1.0
    out /= np.maximum(wgt, 1e-6)
    return out

def write_tiff(arr2d, lat, lon, path, varname="precip_mm"):
    da = xr.DataArray(arr2d, dims=("latitude","longitude"),
                      coords={"latitude": lat, "longitude": lon}, name=varname)
    da = da.rio.write_crs(4326)
    da.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
    da.rio.to_raster(path)

# 1) Pick the wettest test times (by 95th percentile across the grid at lead +1)
tp = test_ds["tp_mmph"]  # [T,H,W]
Tte = tp.sizes["time"]
valid_j_start = CFG["tin"]           # need tin hours before for inputs
valid_j_end   = Tte - 3              # need +3h available
tp_valid = tp.isel(time=slice(valid_j_start, valid_j_end+1))  # times j where +1..+3 exist

# Score per time using P95 (more robust than mean)
scores = tp_valid.quantile(0.95, dim=("latitude","longitude"), skipna=True).values  # [Ntimes]
top_idx_rel = np.argsort(scores)[-TOP_K:][::-1]  # top-K descending
cases_e = (valid_j_start + top_idx_rel - 1).tolist()  # window end index e = j-1

print("Selected rainy window-end indices (e):", cases_e[:10], "... total", len(cases_e))

# 2) Run tiled inference for +1h/+2h/+3h on those cases and export GeoTIFFs
lat_test = test_ds.latitude.values
lon_test = test_ds.longitude.values
tif_dir = Path(OUT_DIR) / "tiffs"
tif_dir.mkdir(parents=True, exist_ok=True)

times_all = []
for ci, e in enumerate(cases_e):
    t_start = e - (CFG["tin"] - 1)
    Xfull = build_input_stack(test_ds, preds_avail, mean_np, std_np, t_start, CFG["tin"])
    preds_leads, truths_leads = [], []
    for L in [1,2,3]:
        y_pred = predict_full(model, Xfull, patch=CFG.get("tile_patch", 192), overlap=CFG.get("tile_overlap", 32))
        preds_leads.append(y_pred)
        y_true = np.asarray(test_ds["tp_mmph"].isel(time=e+L).data, dtype=np.float32)
        truths_leads.append(y_true)
        if L < 3:
            # slide window by 1 hour
            Xfull = np.concatenate([Xfull[:,1:], Xfull[:, -1:]], axis=1)

    for L in [1,2,3]:
        write_tiff(preds_leads[L-1], lat_test, lon_test, tif_dir / f"pred_{ci}_lead{L}.tif")
        write_tiff(truths_leads[L-1], lat_test, lon_test, tif_dir / f"true_{ci}_lead{L}.tif")

    times = pd.to_datetime(test_ds.time.values)
    times_all.append(np.array([times[e+1], times[e+2], times[e+3]], dtype="datetime64[s]"))

np.savez(os.path.join(OUT_DIR, "sample_preds.npz"),
         preds=np.array([0]), truths=np.array([0]),
         times=np.array(times_all), lat=lat_test, lon=lon_test)

print(f"Exported {len(cases_e)} rainy cases to", tif_dir)
gc.collect(); torch.cuda.empty_cache()

Selected rainy window-end indices (e): [81, 46, 80, 79, 47, 78, 45, 44, 72, 82] ... total 12
Exported 12 rainy cases to /content/neural_weather_outputs/tiffs


In [ ]:
import rioxarray as rxr

def build_input_stack(dsX, preds, mean_np, std_np, t_start, tin):
    sel = dsX[preds].isel(time=slice(t_start, t_start+tin)).to_array("channel").transpose("channel","time","latitude","longitude")
    X = np.asarray(sel.data, dtype=np.float32)
    X = (X - mean_np[:, None, None, None]) / std_np[:, None, None, None]

    # Time features → [4,T,H,W]
    times = dsX.time.isel(time=slice(t_start, t_start+tin)).values
    tf = make_time_feats(times)                 # [T,4]
    TF = np.transpose(tf, (1,0))               # [4,T]
    H = dsX.dims["latitude"]; W = dsX.dims["longitude"]
    TF = TF[:, :, None, None]                  # [4,T,1,1]
    TF = np.broadcast_to(TF, (4, tin, H, W)).astype(np.float32)

    return np.concatenate([X, TF], axis=0)     # [C+4, T, H, W]

def predict_full(model, X_full, patch=192, overlap=32):
    C,T,H,W = X_full.shape
    out = np.zeros((H,W), dtype=np.float32)
    wgt = np.zeros((H,W), dtype=np.float32)
    step = patch - overlap
    model.eval()
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
        for r in range(0, H, step):
            for c in range(0, W, step):
                rr = min(r+patch, H); cc = min(c+patch, W)
                r0 = rr - patch if rr - r < patch else r
                c0 = cc - patch if cc - c < patch else c
                x = X_full[:, :, r0:rr, c0:cc][None]
                y = model(torch.from_numpy(x).to(device)).float().cpu().numpy()[0,0]
                y_mm = np.expm1(y).clip(min=0)
                out[r0:rr, c0:cc] += y_mm
                wgt[r0:rr, c0:cc] += 1.0
    out /= np.maximum(wgt, 1e-6)
    return out

def write_tiff(arr2d, lat, lon, path, varname="precip_mm"):
    da = xr.DataArray(arr2d, dims=("latitude","longitude"),
                      coords={"latitude": lat, "longitude": lon}, name=varname)
    da = da.rio.write_crs(4326)
    da.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
    da.rio.to_raster(path)

lat_test = test_ds.latitude.values
lon_test = test_ds.longitude.values
tif_dir = Path(OUT_DIR) / "tiffs"
tif_dir.mkdir(parents=True, exist_ok=True)

Tte = test_ds.time.size
cases_e = [CFG["tin"]-1 + k*6 for k in range(5) if (CFG["tin"]-1 + k*6 + 3) < Tte]
print("Window end indices:", cases_e)

times_all = []
for ci, e in enumerate(cases_e):
    t_start = e - (CFG["tin"] - 1)
    Xfull = build_input_stack(test_ds, preds_avail, mean_np, std_np, t_start, CFG["tin"])
    preds_leads, truths_leads = [], []
    for L in [1,2,3]:
        y_pred = predict_full(model, Xfull, patch=CFG["tile_patch"], overlap=CFG["tile_overlap"])
        preds_leads.append(y_pred)
        y_true = np.asarray(test_ds["tp_mmph"].isel(time=e+L).data, dtype=np.float32)
        truths_leads.append(y_true)
        if L < 3:
            Xfull = np.concatenate([Xfull[:,1:], Xfull[:, -1:]], axis=1)  # slide 1h
    for L in [1,2,3]:
        write_tiff(preds_leads[L-1], lat_test, lon_test, tif_dir / f"pred_{ci}_lead{L}.tif")
        write_tiff(truths_leads[L-1], lat_test, lon_test, tif_dir / f"true_{ci}_lead{L}.tif")
    times = pd.to_datetime(test_ds.time.values)
    times_all.append(np.array([times[e+1], times[e+2], times[e+3]], dtype="datetime64[s]"))

np.savez(os.path.join(OUT_DIR, "sample_preds.npz"),
         preds=np.array([0]), truths=np.array([0]),
         times=np.array(times_all), lat=lat_test, lon=lon_test)

print("Saved GeoTIFFs and sample_preds.npz to", tif_dir)

Window end indices: [5, 11, 17, 23, 29]
Saved GeoTIFFs and sample_preds.npz to /content/neural_weather_outputs/tiffs


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import rasterio
from pathlib import Path
import leafmap.foliumap as leafmap

st.set_page_config(page_title="Neural Weather Forecaster", layout="wide")
st.title("Neural Weather Pattern Forecaster 🌦️")
st.caption("ERA5 India (Apr 2025) — 3D U-Net nowcasting (+1h/+2h/+3h). Rainfall is sparse by nature, use the scale slider to enhance visibility.")

OUT_DIR = Path("/content/neural_weather_outputs")
ASSETS = OUT_DIR / "tiffs"
NPZ = OUT_DIR / "sample_preds.npz"

if not ASSETS.exists():
    st.error("Assets folder missing. Run the export step.")
    st.stop()

cases = sorted(set(p.stem.split("_")[1] for p in ASSETS.glob("pred_*_lead*.tif")))
leads = [1, 2, 3]

# UI
col0, col1, col2 = st.columns([1,1,2])
with col0: case = st.selectbox("Case", options=cases, index=0)
with col1: lead = st.selectbox("Lead (+hours)", options=leads, index=0)

# Determine sensible vmax from truth raster stats
true_path = ASSETS / f"true_{case}_lead{lead}.tif"
pred_path = ASSETS / f"pred_{case}_lead{lead}.tif"
with rasterio.open(true_path) as src:
    arr = src.read(1).astype(np.float32)
    arr[arr <= 0] = np.nan
    p95 = float(np.nanpercentile(arr, 95)) if np.isfinite(np.nanpercentile(arr, 95)) else 10.0
with rasterio.open(pred_path) as src:
    arrp = src.read(1).astype(np.float32)
    arrp[arrp <= 0] = np.nan
    p95p = float(np.nanpercentile(arrp, 95)) if np.isfinite(np.nanpercentile(arrp, 95)) else 10.0

default_vmax = max(5.0, round(max(p95, p95p), 1))
with col2:
    vmax = st.slider("Max scale (mm/h)", min_value=1.0, max_value=100.0, value=min(default_vmax, 30.0), step=1.0)
vmin = 0.0

def center_from_tiff(path):
    with rasterio.open(path) as src:
        b = src.bounds
    return [(b.top + b.bottom) / 2, (b.left + b.right) / 2]

m = leafmap.Map(center=center_from_tiff(true_path), zoom=5, draw_control=False, measure_control=False, fullscreen_control=True)

# Try localtileserver path; else fallback to ImageOverlay
try:
    import localtileserver  # noqa: F401
    HAS_LTS = True
except Exception:
    HAS_LTS = False

if HAS_LTS:
    # Add truth beneath, prediction on top
    m.add_raster(str(true_path), layer_name="Truth (mm/h)", colormap="Blues", vmin=vmin, vmax=vmax, opacity=0.85)
    m.add_raster(str(pred_path), layer_name="Prediction (mm/h)", colormap="Oranges", vmin=vmin, vmax=vmax, opacity=0.60)
else:
    # Lightweight fallback using ImageOverlay with manual scaling
    import folium
    from PIL import Image
    import matplotlib.cm as cm

    def overlay_raster(map_obj, tif_path, layer_name, cmap_name, vmin=0.0, vmax=10.0, opacity=0.8):
        with rasterio.open(tif_path) as src:
            arr = src.read(1).astype(np.float32)
            bounds = [[src.bounds.bottom, src.bounds.left], [src.bounds.top, src.bounds.right]]
            nodata = src.nodata
            mask = ~np.isfinite(arr) | ((arr == nodata) if nodata is not None else False) | (arr <= vmin)
            norm = np.clip((arr - vmin) / (vmax - vmin + 1e-6), 0, 1)
            rgba = (cm.get_cmap(cmap_name)(norm) * 255).astype(np.uint8)
            rgba[mask, 3] = 0
            img = Image.fromarray(rgba, mode="RGBA")
        folium.raster_layers.ImageOverlay(image=img, bounds=bounds, name=layer_name, opacity=opacity, interactive=True).add_to(map_obj)

    overlay_raster(m, str(true_path), "Truth (mm/h)", "Blues", vmin=vmin, vmax=vmax, opacity=0.85)
    overlay_raster(m, str(pred_path), "Prediction (mm/h)", "Oranges", vmin=vmin, vmax=vmax, opacity=0.60)

m.add_layer_control()
st.subheader("Interactive map")
m.to_streamlit(height=650)

# Timestamp if saved
if NPZ.exists():
    dat = np.load(NPZ, allow_pickle=True)
    times = dat["times"]
    try:
        trow = int(case)
        tval = times[trow][lead - 1]
        st.caption(f"Valid time: {np.datetime_as_string(tval, unit='s')}")
    except Exception:
        pass

st.markdown("---")
st.write("Tip: Increase 'Max scale (mm/h)' to highlight heavier rain. Try different cases; we picked the wettest hours to ensure visible patterns.")

Overwriting app.py


In [ ]:
# Stop any previous app
!pkill -f "streamlit run app.py" || true

# Launch Streamlit with ngrok
import subprocess, os
from pyngrok import ngrok
from getpass import getpass

port = 8501
proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", str(port), "--server.address", "0.0.0.0"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN") or getpass("Paste your ngrok auth token: ")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(port, "http")
print("Your Streamlit app is live at:", public_url)

^C
Paste your ngrok auth token: ··········
Your Streamlit app is live at: NgrokTunnel: "https://chololithic-myrtle-fernless.ngrok-free.dev" -> "http://localhost:8501"
